In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from ggplot import *


In [3]:
#create dataframe
turnstile_weather = pd.read_csv('~/Documents/ds_nanodegree/intro_to_ds_downloads/turnstile_weather_v2.csv')



In [4]:
def normalize_features(features):
    ''' 
    Returns the means and standard deviations of the given features, along with a normalized feature
    matrix.
    ''' 
    means = np.mean(features, axis=0)
    std_devs = np.std(features, axis=0)
    normalized_features = (features - means) / std_devs
    
    #return means, std_devs, normalized_features
    return normalized_features



def linear_regression(features, values):
    """
    Perform linear regression given a data set with an arbitrary number of features.
    
    This can be the same code as in the lesson #3 exercise.
    """
    
    features = sm.add_constant(features)
    model = sm.OLS(values,features)
    results = model.fit()
    params = results.params[1:]
    intercept = results.params[0]
    
    #return intercept, params
    return intercept, params
    
    #return summary
    #return model.fit().summary()
    
    #return various measures can be extracted from results
    #return results.mse_resid


def predictions(dataframe): 
    
    '''Before using this, fix the linear_regression function above
       so it returns const & params, instead of OLS summary'''
    
    #define input variables
    features = dataframe[['tempi']]

    #add dummy vars
    dummy_units = pd.get_dummies(dataframe['UNIT'], prefix='unit')
    dummy_day_week = pd.get_dummies(dataframe['day_week'], prefix='day')
    dummy_hours = pd.get_dummies(dataframe['hour'], prefix='hour')

    features = features.join([dummy_units,dummy_hours, dummy_day_week])

    #normalize
    features = normalize_features(features)

    #drop a unit to fix multicollinearity
    features.drop(['unit_R022','hour_0','day_0'],axis=1,inplace=True)
        
    # Values
    values = dataframe['ENTRIESn_hourly']

    # Perform linear regression
    intercept, params = linear_regression(features, values)
    
    predictions = intercept + np.dot(features, params)
    return predictions



In [8]:
# R-squared

#For this to work, first you need to uncomment 
#the correct return statement in the linear_regression function.
#Then unmcomment the below.

def compute_r_squared(data, predictions):

    accuracy = ((data - predictions)**2).sum()
    var = ((data - data.mean())**2).sum()
    r_squared = 1 - (accuracy/var)
    
    return r_squared

compute_r_squared(turnstile_weather['ENTRIESn_hourly'], predictions(turnstile_weather))

0.54526423980394689

In [5]:
#OLS summary
#TODO: remove duplication of "features"

#define input variables
features = turnstile_weather[['tempi']]

#add dummy vars
dummy_units = pd.get_dummies(turnstile_weather['UNIT'], prefix='unit')
dummy_day_week = pd.get_dummies(turnstile_weather['day_week'], prefix='day')
dummy_hours = pd.get_dummies(turnstile_weather['hour'], prefix='hour')

features = features.join([dummy_units,dummy_hours, dummy_day_week])

#normalize
features = normalize_features(features)

#drop a unit to fix multicollinearity
features.drop(['unit_R022','hour_0','day_0'],axis=1,inplace=True)
        
# Values
values = turnstile_weather['ENTRIESn_hourly']

#can switch this function so it returns summary instead of predicted values
linear_regression(features,turnstile_weather['ENTRIESn_hourly'])


(1886.5899552157894, tempi       -119.839284
 unit_R003   -591.232124
 unit_R004   -580.222090
 unit_R005   -576.140015
 unit_R006   -577.801795
 unit_R007   -584.554580
 unit_R008   -582.974118
 unit_R009   -593.090467
 unit_R011   -143.272944
 unit_R012    -54.951533
 unit_R013   -457.014230
 unit_R016   -574.094613
 unit_R017   -350.594038
 unit_R018   -114.802929
 unit_R019   -414.142931
 unit_R020   -207.199978
 unit_R021   -316.850653
 unit_R023   -221.725236
 unit_R024   -415.542622
 unit_R025   -275.968045
 unit_R027   -431.638249
 unit_R029   -150.795211
 unit_R030   -422.929116
 unit_R031   -340.443716
 unit_R032   -333.312983
 unit_R033    -84.572432
 unit_R034   -529.323596
 unit_R035   -440.508981
 unit_R036   -565.843140
 unit_R037   -567.235134
                 ...    
 unit_R345   -578.402098
 unit_R346   -526.864790
 unit_R348   -599.536464
 unit_R354   -585.398391
 unit_R356   -540.536048
 unit_R358   -583.245742
 unit_R370   -579.595346
 unit_R371   -559.252438
 unit

In [12]:
#add predictions into new data frame
actual = turnstile_weather['ENTRIESn_hourly']
predicted = predictions(turnstile_weather)

turnstile_weather['x'] = turnstile_weather.index
new_df = turnstile_weather
new_df['predicted'] = predicted
new_df['actual'] = actual
new_df['residual'] = actual - predicted

In [ ]:
#How well does this model work?
#compare actual to predicted entries

compare_df = pd.melt(new_df, id_vars=['x'], value_vars=['actual', 'predicted'], var_name='val_type', value_name='values')

ggplot(compare_df, aes(x='x',y='values', color='val_type')) +\
       geom_point(size=.5, alpha=.7) +\
       xlab('Remote unit readings') +\
       ylab('Entries') +\
       ggtitle("Actual vs Predicted Hourly Entries")




In [21]:
#Plot the residuals against fitted values
fit_resid_plot = ggplot(new_df, aes('predicted','residual')) +\
                geom_point(size=.5, colour = 'red') +\
                ggtitle('Residuals vs Predicted Values')
fit_resid_plot

<ggplot: (286238853)>

In [28]:
#model checking: is std dev of model = std dev of residuals? 
print np.std(new_df['predicted'])
print np.std(new_df['residual'])
print np.std(new_df['actual'])

2180.07529143
1990.89125409
2952.35097201


In [26]:
#plot residuals against data points
points_resid = ggplot(new_df, aes('x', 'residual')) +\
                geom_point(size = 3, colour = 'blue') +\
    ggtitle('Residuals vs Data Points')

points_resid

<ggplot: (286180993)>

In [16]:
#plot residuals against independant variable
#temperature

tempi_resid = ggplot(new_df, aes('tempi', 'residual')) +\
                geom_point(size = 1, colour = 'red') +\
    ggtitle('Residuals vs Temperature (Independant Variable)')

tempi_resid


<ggplot: (285353713)>

In [16]:
#day of week
day_resid = ggplot(new_df, aes('day_week', 'residual')) +\
                geom_point(size = 3, colour = 'red') +\
    ggtitle('Residuals vs Day of Week (Independant Variable)')

day_resid

<ggplot: (284317837)>

In [17]:
#hour of day
hour_resid = ggplot(new_df, aes('hour', 'residual')) +\
                geom_point(size = 3, colour = 'red') +\
    ggtitle('Residuals vs Hour of Day (Independant Variable)')

hour_resid

<ggplot: (282876025)>

In [19]:
#remote unit
#convert to float-able

ru = new_df['UNIT']
ru = ru.str.replace('R','')
new_df['ru'] = ru

unit_resid = ggplot(new_df, aes('ru', 'residual')) +\
                geom_point(size = 3, colour = 'red') +\
    ggtitle('Residuals vs Remote Unit (Independant Variable)')

unit_resid

<ggplot: (284512329)>

In [42]:
#Are the residuals a normal distribution?

import scipy
import matplotlib.pyplot as plt



plt.figure()
(new_df['residual']).hist(bins=150, label = 'Residuals', color='red', edgecolor = 'blue')
plt.draw()
plt.show()



In [23]:
#Better way to check distribution of residuals: use a probability plot
import pylab 
import scipy.stats as stats

np.random.seed(124)

measurements = new_df['residual']  
#sample = np.random.choice(measurements, size=10000, replace=True, p=None)
#measurements = np.random.normal(loc = 20, scale = 5, size=100)   
stats.probplot(measurements, dist="norm", plot=pylab)
pylab.show()